In [2]:
#Author: nhny at hcmut.edu.vn
#Course: AI 2016
#Lab 3
# DFS, BFS, Hill Climbing, A-Star with heuristics manhattan distance and hamming distance. 

import heapq

class Node(object):
    """
    Represent state of board in 8 puzzle problem.
    """
    n = 0

    def __init__(self, board, prev_state = None):
        assert len(board) == 9

        self.board = board[:];
        self.prev = prev_state
        self.step = 0
        Node.n += 1

        if self.prev:
            self.step = self.prev.step + 1 

    def __eq__(self, other):
        """Check wether two state is equal."""
        return self.board == other.board

    def __hash__(self):
        """Return hash code of object.

        Used for comparing elements in set
        """
        h = [0, 0, 0]
        h[0] = self.board[0] << 6 | self.board[1] << 3 | self.board[2]
        h[1] = self.board[3] << 6 | self.board[4] << 3 | self.board[5]
        h[2] = self.board[6] << 6 | self.board[7] << 3 | self.board[8]

        h_val = 0
        for h_i in h:
            h_val = h_val * 31 + h_i

        return h_val

    def __str__(self):
        string_list = [str(i) for i in self.board]
        sub_list = (string_list[:3], string_list[3:6], string_list[6:])
        return "\n".join([" ".join(l) for l in sub_list])

    def manhattan_distance(self):
        """Return Manhattan distance of state."""
        #TODO: return Manhattan distance
        distance = 0
        goal = [1,2,3,4,5,6,7,8,0]
        for i in range(1,9):
            xs, ys = self.__i2pos(self.board.index(i))
            xg, yg = self.__i2pos(goal.index(i))
            distance += abs(xs-xg) + abs(ys-yg)
        return distance

    def manhattan_score(self):
        """Return Manhattan score of state."""
        #TODO: return Manhattan score of state
        return 0

    def hamming_distance(self):
        """Return Hamming distance of state."""
        #TODO: return Hamming distance
        distance = 0
        goal = [1,2,3,4,5,6,7,8,0]
        for i in range(9):
            if goal[i] != self.board[i]: distance += 1
        return distance

    def hamming_score(self):
        """Return Hamming distance score of state."""
        #TODO return Hamming score of state
        return 0

    def next(self):
        """Return next states from this state."""
        next_moves = []
        i = self.board.index(0)

        next_moves = (self.move_up(i), self.move_down(i), self.move_left(i), self.move_right(i))

        return [s for s in next_moves if s]

    def move_right(self, i):
        x, y = self.__i2pos(i)
        if y < 2:
            right_state = Node(self.board, self)
            right = self.__pos2i(x, y+1)
            right_state.__swap(i, right)
            return right_state

    def move_left(self, i):
        x, y = self.__i2pos(i)
        if y > 0:
            left_state = Node(self.board, self)
            left = self.__pos2i(x, y - 1)
            left_state.__swap(i, left)
            return left_state

    def move_up(self, i):
        x, y = self.__i2pos(i)
        if x > 0:
            up_state = Node(self.board, self)
            up = self.__pos2i(x - 1, y)
            up_state.__swap(i, up)
            return up_state

    def move_down(self, i):
        x, y = self.__i2pos(i)
        if x < 2:
            down_state = Node(self.board, self)
            down = self.__pos2i(x + 1, y)
            down_state.__swap(i, down)
            return down_state

    def __swap(self, i, j):
        self.board[j], self.board[i] = self.board[i], self.board[j]

    def __i2pos(self, index):
        return (int(index / 3), index % 3)

    def __pos2i(self, x, y):
        return x * 3 + y


class PriorityQueue:
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        # FIXME: restored old behaviour to check against old results better
        # FIXED: restored to stable behaviour
        entry = (priority, self.count, item)
        # entry = (priority, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        #  (_, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

In [3]:
#!/usr/bin/env python

#Author: nhny at hcmut.edu.vn
#Course: AI 2016
#Lab 3
from sys import argv
from time import time
from node import Node
from node import PriorityQueue

class Searcher(object):
    """Searcher that manuplate searching process."""

    def __init__(self, start, goal):
        self.start = start
        self.goal = goal

    def print_path(self, state):
        path = []
        while state:
            path.append(state)
            state = state.prev
        path.reverse()
        print("\n-->\n".join([str(state) for state in path]))

    def dfs(self, depth = 100):
        stack = [self.start]
        visited = set()
        found = False

        while stack:
            state = stack.pop()

            if state == self.goal:
                found = state
                break

            if state in visited or state.step > depth:
                continue

            visited.add(state)

            for s in state.next():
                stack.append(s)

        if found:
            self.print_path(found)
            print("Find solution")
        else:
            print("No solution found")

    def bfs(self, depth = 50):
        """Run Breadth-first search."""
        #TODO: Implement breadth first search
        queue = [self.start]
        visited = set()
        found = False

        while queue:
            state = queue.pop()

            if state == self.goal:
                found = state
                break

            if state in visited or state.step > depth:
                continue

            visited.add(state)

            for s in state.next():
                queue.insert(0, s)

        if found:
            self.print_path(found)
            print("Find solution")
        else:
            print("No solution found")

    def steepest_ascent_hill_climbing(self):
        """Run steepest ascent hill climbing search."""
        #TODO Implement hill climbing.
        stack = [self.start]

        while stack:
            state = stack.pop()
            if state == self.goal:
                self.print_path(state)
                print("Find solution")
                break

            h_val = state.manhattan_distance() + state.hamming_distance()
            next_state = False
            for s in state.next():
                h_val_next = s.manhattan_distance() + s.hamming_distance()
                if h_val_next < h_val:
                    next_state = s
                    h_val = h_val_next

            if next_state: 
                stack.append(next_state)
            else:
                self.print_path(state)
                print("Cannot find solution")


    # I don't know this function
    def hill_climbing(self):
        """Run hill climbing search."""
        #TODO Implement hill climbing.
        stack = [self.start]

        while stack:
            state = stack.pop()
            if state == self.goal:
                self.print_path(state)
                print("Find solution")
                break

            h_val = state.manhattan_distance() + state.hamming_distance()
            next_state = False
            for s in state.next():
                h_val_next = s.manhattan_distance() + s.hamming_distance()
                if h_val_next < h_val:
                    next_state = s
                    h_val = h_val_next
                    stack.append(next_state)
                    break

            if not next_state:
                self.print_path(state)
                print("Cannot find solution")
            
    def astar(self, depth = 75):
        """Run A* search."""
        #TODO: Implement a star search.
        priotity_queue = PriorityQueue()
        h_val = self.start.manhattan_distance() + self.start.hamming_distance()
        # g_val always is start.step
        f_val = h_val + self.start.step
        priotity_queue.push(self.start, f_val)
        visited = set()
        found = False

        while not priotity_queue.isEmpty():
            state = priotity_queue.pop()

            if state == self.goal:
                found = state
                break

            if state in visited or state.step > depth:
                continue

            visited.add(state)

            for s in state.next():
                h_val_s = s.manhattan_distance() + s.hamming_distance()
                f_val_s = h_val_s + s.step
                priotity_queue.push(s, f_val_s)

        if found:
            self.print_path(found)
            print("Find solution")
        else:
            print("No solution found")


if __name__ == "__main__":
    script = "astar"
    strategy = "astar"

    #Unit test
    print("Search for solution\n")
    start = Node([2,4,3,1,0,6,7,5,8])
    goal = Node([1,2,3,4,5,6,7,8,0])

    print(start.hamming_distance())
    print(start.manhattan_distance())

    search = Searcher(start, goal)

    start_time = time()
    if strategy == "dfs":
        search.dfs()
    elif strategy == "bfs":
        search.bfs()
    elif strategy == "hc":
        search.hill_climbing()
    elif strategy == "sahc":
        search.steepest_ascent_hill_climbing()
    elif strategy == "astar":
        search.astar()
    else:
        print("Wrong strategy")
    end_time = time()
    elapsed = end_time - start_time
    print("Search time: %s" % elapsed)
    print("Number of initialized node: %d" % Node.n)

ModuleNotFoundError: No module named 'node'